In [1]:
%matplotlib inline
import importlib
import numpy as np
import pandas as pd
import json
import utils

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/encoder-decoder/utils.py'>

In [3]:
dat = pd.read_csv('../songdata.csv', usecols=['text'], nrows=100).drop_duplicates()

In [4]:
dat.head()

,text
0,"Look at her face, it's a wonderful face \nAnd..."
1,"Take it easy with me, please \nTouch me gentl..."
2,I'll never know why I had to go \nWhy I had t...
3,Making somebody happy is a question of give an...
4,Making somebody happy is a question of give an...


In [5]:
glove = pd.read_table('../glove.6B.50d.txt', sep=' ', header=None, quoting=3, na_filter=False, index_col=0)

In [6]:
dat.iloc[0, 0]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?  \n  \nAnd when we go for a walk in the park  \nAnd she holds me and squeezes my hand  \nWe'll go on walking for hours and talking  \nAbout all the things that we plan  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?\n\n"

In [7]:
dat.loc[:, 'text'] += '(end)'

In [38]:
dat.loc[:, 'cleaned_text'] = utils.data_curating(dat.copy())

In [39]:
cleaned = dat.cleaned_text.str.split(' ')
cleaned_sets = cleaned.apply(set)
vocab_size, unique_tokens = utils.get_unique_tokens(cleaned_sets)

In [40]:
dat.loc[:, 'cleaned_text'] = dat.cleaned_text.str.split('\n')

In [41]:
dat.loc[:, 'cleaned_text'] = dat.cleaned_text.apply(lambda sens: [sen.strip() for sen in sens])

In [42]:
dat.cleaned_text.head()

0    [look at her face , it 's a wonderful face, an...
1    [take it easy with me , please, touch me gentl...
2    [i 'll never know why i had to go, why i had t...
3    [making somebody happy is a question of give a...
4    [making somebody happy is a question of give a...
Name: cleaned_text, dtype: object

In [90]:
encoder_input = dat.cleaned_text.apply(lambda x: x[:-1])

In [91]:
decoder_input = dat.cleaned_text.apply(lambda x: x[1:])

In [92]:
print(decoder_input[0])

['and it means something special to me', 'look at the way that she smiles when she sees me', 'how lucky can one fellow be ?', '', "she 's just my kind of girl , she makes me feel fine", 'who could ever believe that she could be mine ?', "she 's just my kind of girl , without her i 'm blue", 'and if she ever leaves me what could i do , what could i do ?', '', 'and when we go for a walk in the park', 'and she holds me and squeezes my hand', "we 'll go on walking for hours and talking", 'about all the things that we plan', '', "she 's just my kind of girl , she makes me feel fine", 'who could ever believe that she could be mine ?', "she 's just my kind of girl , without her i 'm blue", 'and if she ever leaves me what could i do , what could i do ?', '', '( end )']


In [93]:
encoder_input = encoder_input.apply(lambda sens: [sen.split(' ') for sen in sens])
decoder_input = decoder_input.apply(lambda sens: [sen.split(' ') for sen in sens])

In [94]:
print(encoder_input[0])

[['look', 'at', 'her', 'face', ',', 'it', "'s", 'a', 'wonderful', 'face'], ['and', 'it', 'means', 'something', 'special', 'to', 'me'], ['look', 'at', 'the', 'way', 'that', 'she', 'smiles', 'when', 'she', 'sees', 'me'], ['how', 'lucky', 'can', 'one', 'fellow', 'be', '?'], [''], ['she', "'s", 'just', 'my', 'kind', 'of', 'girl', ',', 'she', 'makes', 'me', 'feel', 'fine'], ['who', 'could', 'ever', 'believe', 'that', 'she', 'could', 'be', 'mine', '?'], ['she', "'s", 'just', 'my', 'kind', 'of', 'girl', ',', 'without', 'her', 'i', "'m", 'blue'], ['and', 'if', 'she', 'ever', 'leaves', 'me', 'what', 'could', 'i', 'do', ',', 'what', 'could', 'i', 'do', '?'], [''], ['and', 'when', 'we', 'go', 'for', 'a', 'walk', 'in', 'the', 'park'], ['and', 'she', 'holds', 'me', 'and', 'squeezes', 'my', 'hand'], ['we', "'ll", 'go', 'on', 'walking', 'for', 'hours', 'and', 'talking'], ['about', 'all', 'the', 'things', 'that', 'we', 'plan'], [''], ['she', "'s", 'just', 'my', 'kind', 'of', 'girl', ',', 'she', 'makes

In [95]:
encoder_input = encoder_input.sum()

In [96]:
encoder_input_data = []

for sen in encoder_input:
    out = [' '] * 30
    for ind, word in enumerate(sen):
        out[ind] = word if word != '' else ' '
    encoder_input_data.append(out)

encoder_input_data = pd.Series(encoder_input_data)

In [97]:
decoder_input = decoder_input.sum()

In [98]:
decoder_input[0]

['and', 'it', 'means', 'something', 'special', 'to', 'me']

In [100]:
decoder_input_data = []

for sen in decoder_input:
    out = [' '] * 25
    for ind, word in enumerate(sen):
        out[ind] = word if word != '' else ' '
    decoder_input_data.append(out)

decoder_input_data = pd.Series(decoder_input_data)

In [50]:
vocab_size

In [52]:
unique_tokens[:5]

['\n', '!', '"', "'", "'cause"]

In [53]:
unique_tokens.insert(0, ' ')

In [54]:
unique_tokens[:5]

[' ', '\n', '!', '"', "'"]

In [55]:
utils.save_unique_tokens(unique_tokens)

In [56]:
word2ind, ind2word = utils.get_index_word_map(unique_tokens)

In [57]:
utils.save_index_word_map(word2ind, ind2word)

In [58]:
emb = utils.create_emb(ind2word, glove)

In [59]:
utils.save_embedding(emb)

In [62]:
encoder_input_tokenised = utils.tokenise_cleaned_data(encoder_input_data, word2ind)

In [63]:
encoder_input_tokenised.head()

0    [1102, 120, 896, 636, 19, 985, 13, 28, 2103, 6...
1    [86, 985, 1163, 1689, 1708, 1889, 1161, 0, 0, ...
2    [1102, 120, 1851, 2044, 1850, 1605, 1671, 2064...
3    [943, 1131, 282, 1290, 676, 156, 25, 0, 0, 0, ...
4    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
dtype: object

In [158]:
decoder_input_tokenised = utils.tokenise_cleaned_data(decoder_input_data, word2ind)

In [159]:
decoder_input_tokenised.head()

0    [86, 985, 1163, 1689, 1708, 1889, 1161, 0, 0, ...
1    [1102, 120, 1851, 2044, 1850, 1605, 1671, 2064...
2    [943, 1131, 282, 1290, 676, 156, 25, 0, 0, 0, ...
3    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [1605, 13, 1004, 1232, 1017, 1279, 777, 19, 16...
dtype: object

In [160]:
len(decoder_input_tokenised.loc[0])

25

In [161]:
decoder_target_tokenised = decoder_input_tokenised.apply(lambda x: x + [word2ind['\n']])

In [162]:
decoder_target_tokenised.head()

0    [86, 985, 1163, 1689, 1708, 1889, 1161, 0, 0, ...
1    [1102, 120, 1851, 2044, 1850, 1605, 1671, 2064...
2    [943, 1131, 282, 1290, 676, 156, 25, 0, 0, 0, ...
3    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [1605, 13, 1004, 1232, 1017, 1279, 777, 19, 16...
dtype: object

In [163]:
len(decoder_target_tokenised.loc[0])

26

In [164]:
decoder_input_tokenised = decoder_input_tokenised.apply(lambda x: [word2ind[' ']] + x)

In [165]:
decoder_input_tokenised.head()

0    [0, 86, 985, 1163, 1689, 1708, 1889, 1161, 0, ...
1    [0, 1102, 120, 1851, 2044, 1850, 1605, 1671, 2...
2    [0, 943, 1131, 282, 1290, 676, 156, 25, 0, 0, ...
3    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 1605, 13, 1004, 1232, 1017, 1279, 777, 19,...
dtype: object

In [166]:
len(decoder_input_tokenised.iloc[0])

26

In [167]:
encoder_input_tokenised = pd.DataFrame(encoder_input_tokenised.values.tolist())
decoder_input_tokenised = pd.DataFrame(decoder_input_tokenised.values.tolist())
decoder_target_tokenised = pd.DataFrame(decoder_target_tokenised.values.tolist())

In [168]:
encoder_input_tokenised.shape

(4462, 25)

In [169]:
decoder_input_tokenised.shape

(4462, 26)

In [170]:
decoder_target_tokenised.shape

(4462, 26)

In [171]:
encoder_input_tokenised.to_csv('encoder_input.csv', index=False)
decoder_input_tokenised.to_csv('decoder_input.csv', index=False)
decoder_target_tokenised.to_csv('decoder_target.csv', index=False)